In [1]:
import utilities as utils

/home/drake/miniconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index



    Last Execution: 2022-04-28 15:59:34.034533
    python:	3.8.5

    	fastai:		2.5.6
    	matplotlib:	3.5.1
    	numpy:		1.19.5
    	pandas:		1.4.1
    	seaborn:	0.11.2
    	sklearn:	1.0.2
    	torch:		1.11.0+cu102
    	yellowbrick:	1.4
    	imblearn:	0.9.0
    


In [14]:
data_path_1: str = '../../../Data/phase2'   
data_path_2: str = '../../../Data/phase1/'  
 

data_set_1: list = [ 'vae_application_dataset.csv', 'vae_traffic_dataset.csv', ]

data_set_2: list = [ 'Traffic_type_seed.csv', 'Application_type_seed.csv' ] 


file_path_1       = utils.get_file_path(data_path_1)
file_path_2       = utils.get_file_path(data_path_2)
file_set_1 : list = list(map(file_path_1, data_set_1))
file_set_2 : list = list(map(file_path_2, data_set_2))

file_set : list   = file_set_1 + file_set_2 
data_set   : list = data_set_1 + data_set_2 
current_job: int  = 0

utils.data_set = data_set
utils.file_set = file_set

In [15]:
print(f'We will be using {len(file_set)} files:')
utils.pretty(file_set)

We will be using 4 files:
[   '../../../Data/phase2/vae_application_dataset.csv',
    '../../../Data/phase2/vae_traffic_dataset.csv',
    '../../../Data/phase1/Traffic_type_seed.csv',
    '../../../Data/phase1/Application_type_seed.csv']


In [16]:
vae_application_dataset_labels_50 = utils.examine_dataset(1)
vae_traffic_dataset_100 = utils.examine_dataset(2)
baseline_traffic_seed = utils.examine_dataset(3)
baseline_application_seed = utils.examine_dataset(4)

vae_traffic_dataset_100['Dataset'] = vae_traffic_dataset_100['Dataset'].drop(['Unnamed: 0'], axis = 1)
vae_application_dataset_labels_50['Dataset'] = vae_application_dataset_labels_50['Dataset'].drop(['Unnamed: 0'], axis = 1)

Dataset 1/4: We now look at ../../../Data/phase2/vae_application_dataset.csv


Loading Dataset: ../../../Data/phase2/vae_application_dataset.csv
	To Dataset Cache: ./cache/vae_application_dataset.csv.pickle


        File:				../../../Data/phase2/vae_application_dataset.csv  
        Job Number:			1
        Shape:				(400000, 65)
        Samples:			400000 
        Features:			65
    
Dataset 2/4: We now look at ../../../Data/phase2/vae_traffic_dataset.csv


Loading Dataset: ../../../Data/phase2/vae_traffic_dataset.csv
	To Dataset Cache: ./cache/vae_traffic_dataset.csv.pickle


        File:				../../../Data/phase2/vae_traffic_dataset.csv  
        Job Number:			2
        Shape:				(300000, 65)
        Samples:			300000 
        Features:			65
    
Dataset 3/4: We now look at ../../../Data/phase1/Traffic_type_seed.csv


Loading Dataset: ../../../Data/phase1/Traffic_type_seed.csv
	To Dataset Cache: ./cache/Traffic_type_seed.csv.pickle


        File:				../../../Data/phase1/Traffic_type_

In [17]:
def downsample(df: utils.pd.DataFrame, column_name: str, expected_sizes: dict) -> utils.pd.DataFrame():
    '''
        Function returns a new dataframe with the given column name and value
    '''
    new_dict = utils.pd.DataFrame()
    for item in df[column_name].unique():
        matching_values = df.loc[df[column_name] == item]
        if df[column_name].value_counts()[item] > expected_sizes[item]:
            new_dict = utils.pd.concat([new_dict, matching_values.sample(n = expected_sizes[item])])
        else:
            new_dict = utils.pd.concat([new_dict, matching_values])
    return new_dict

def random_sample(df: utils.pd.DataFrame, column_name: str, element_name: str, size : int) -> utils.pd.DataFrame():
    '''
        Function returns a new dataframe with the given column name and value
    '''
    new_df = df.loc[df[column_name] == element_name]
    return new_df.sample(n = size)

# Traffic Type Datasets Creation

In [18]:
expected_sizes = {"Regular" : 30000, "VPN" : 20000, "Tor": 10000}
vae_balanced_traffic_labels_dataset_30_20_10 = downsample(baseline_traffic_seed['Dataset'], 'Traffic Type', expected_sizes)
vae_balanced_traffic_labels_dataset_30_20_10 = utils.pd.concat([vae_balanced_traffic_labels_dataset_30_20_10, random_sample(vae_traffic_dataset_100['Dataset'], 'Traffic Type', 'Tor', 10000 - baseline_traffic_seed['Dataset']['Traffic Type'].value_counts()['Tor'])])

In [19]:
expected_sizes = {"Regular" : 92659, "VPN" : 92659, "Tor": 92659}
vae_balanced_traffic_labels_dataset_equal = downsample(baseline_traffic_seed['Dataset'], 'Traffic Type', expected_sizes)
vae_balanced_traffic_labels_dataset_equal = utils.pd.concat([vae_balanced_traffic_labels_dataset_equal, random_sample(vae_traffic_dataset_100['Dataset'], 'Traffic Type', 'VPN', 92659 - baseline_traffic_seed['Dataset']['Traffic Type'].value_counts()['VPN'])])
vae_balanced_traffic_labels_dataset_equal = utils.pd.concat([vae_balanced_traffic_labels_dataset_equal, random_sample(vae_traffic_dataset_100['Dataset'], 'Traffic Type', 'Tor', 92659 - baseline_traffic_seed['Dataset']['Traffic Type'].value_counts()['Tor'])])

# Application Types Data Creation

In [20]:
expected_sizes = {"p2p" : 30000, "browsing" : 30000, "audio-streaming": 30000, 'file-transfer' : 30000, 'chat': 30000, 'video-streaming': 30000, 'voip': 30000, 'email': 30000}
vae_balanced_application_dataset_labels_30_30_30 = downsample(baseline_application_seed['Dataset'], 'Application Type', expected_sizes)
vae_balanced_application_dataset_labels_30_30_30 = utils.pd.concat([vae_balanced_application_dataset_labels_30_30_30, random_sample(vae_application_dataset_labels_50['Dataset'], 'Application Type', 'audio-streaming', 30000 - baseline_application_seed['Dataset']['Application Type'].value_counts()['audio-streaming'])])
vae_balanced_application_dataset_labels_30_30_30 = utils.pd.concat([vae_balanced_application_dataset_labels_30_30_30, random_sample(vae_application_dataset_labels_50['Dataset'], 'Application Type', 'file-transfer', 30000 - baseline_application_seed['Dataset']['Application Type'].value_counts()['file-transfer'])])
vae_balanced_application_dataset_labels_30_30_30 = utils.pd.concat([vae_balanced_application_dataset_labels_30_30_30, random_sample(vae_application_dataset_labels_50['Dataset'], 'Application Type', 'chat', 30000 - baseline_application_seed['Dataset']['Application Type'].value_counts()['chat'])])
vae_balanced_application_dataset_labels_30_30_30 = utils.pd.concat([vae_balanced_application_dataset_labels_30_30_30, random_sample(vae_application_dataset_labels_50['Dataset'], 'Application Type', 'video-streaming', 30000 - baseline_application_seed['Dataset']['Application Type'].value_counts()['video-streaming'])])
vae_balanced_application_dataset_labels_30_30_30 = utils.pd.concat([vae_balanced_application_dataset_labels_30_30_30, random_sample(vae_application_dataset_labels_50['Dataset'], 'Application Type', 'voip', 30000 - baseline_application_seed['Dataset']['Application Type'].value_counts()['voip'])])
vae_balanced_application_dataset_labels_30_30_30 = utils.pd.concat([vae_balanced_application_dataset_labels_30_30_30, random_sample(vae_application_dataset_labels_50['Dataset'], 'Application Type', 'email', 30000 - baseline_application_seed['Dataset']['Application Type'].value_counts()['email'])])


In [21]:
expected_sizes = {"p2p" : 48020, "browsing" : 48020, "audio-streaming": 48020, 'file-transfer' : 48020, 'chat': 48020, 'video-streaming': 48020, 'voip': 48020, 'email': 48020}
vae_balanced_application_dataset_labels_equal = downsample(baseline_application_seed['Dataset'], 'Application Type', expected_sizes)
vae_balanced_application_dataset_labels_equal = utils.pd.concat([vae_balanced_application_dataset_labels_equal, random_sample(vae_application_dataset_labels_50['Dataset'], 'Application Type', 'audio-streaming', 48020 - baseline_application_seed['Dataset']['Application Type'].value_counts()['audio-streaming'])])
vae_balanced_application_dataset_labels_equal = utils.pd.concat([vae_balanced_application_dataset_labels_equal, random_sample(vae_application_dataset_labels_50['Dataset'], 'Application Type', 'file-transfer', 48020 - baseline_application_seed['Dataset']['Application Type'].value_counts()['file-transfer'])])
vae_balanced_application_dataset_labels_equal = utils.pd.concat([vae_balanced_application_dataset_labels_equal, random_sample(vae_application_dataset_labels_50['Dataset'], 'Application Type', 'chat', 48020 - baseline_application_seed['Dataset']['Application Type'].value_counts()['chat'])])
vae_balanced_application_dataset_labels_equal = utils.pd.concat([vae_balanced_application_dataset_labels_equal, random_sample(vae_application_dataset_labels_50['Dataset'], 'Application Type', 'video-streaming', 48020 - baseline_application_seed['Dataset']['Application Type'].value_counts()['video-streaming'])])
vae_balanced_application_dataset_labels_equal = utils.pd.concat([vae_balanced_application_dataset_labels_equal, random_sample(vae_application_dataset_labels_50['Dataset'], 'Application Type', 'voip', 48020 - baseline_application_seed['Dataset']['Application Type'].value_counts()['voip'])])
vae_balanced_application_dataset_labels_equal = utils.pd.concat([vae_balanced_application_dataset_labels_equal, random_sample(vae_application_dataset_labels_50['Dataset'], 'Application Type', 'email', 48020 - baseline_application_seed['Dataset']['Application Type'].value_counts()['email'])])
vae_balanced_application_dataset_labels_equal = utils.pd.concat([vae_balanced_application_dataset_labels_equal, random_sample(vae_application_dataset_labels_50['Dataset'], 'Application Type', 'browsing', 48020 - baseline_application_seed['Dataset']['Application Type'].value_counts()['browsing'])])



In [23]:
vae_balanced_traffic_labels_dataset_30_20_10.to_csv('./synthetic/vae_traffic_30_20_10.csv', index=False)
vae_balanced_traffic_labels_dataset_equal.to_csv('./synthetic/vae_traffic_upsample_to_majority.csv', index=False)
vae_balanced_application_dataset_labels_30_30_30.to_csv('./synthetic/vae_application_30000.csv', index=False)
vae_balanced_application_dataset_labels_equal.to_csv('./synthetic/vae_application_upsample_to_majority.csv', index=False)

In [24]:
print(f'Last Execution: {utils.datetime.datetime.now()}')
assert False, 'Nothing after this point is included in the study'

Last Execution: 2022-04-28 16:12:22.075012


AssertionError: Nothing after this point is included in the study